In [8]:
import pandas as pd
from pathlib import Path

# ───────────────────────────────────────── bottom 데이터 → long
feat_path = Path("data/feature_store/daily_features_per_new.feather")
df = pd.read_feather(feat_path)

# ▶︎ 날짜·정렬은 이미 돼 있다고 가정
df["date"] = pd.to_datetime(df["date"])


In [12]:
df['utilization_7'].unique()

array([0.        , 1.        , 0.5       , 2.        , 0.33333333,
       0.16666667, 3.        , 4.        , 2.5       , 1.66666667,
       0.2       , 0.66666667, 0.4       , 1.5       , 0.75      ,
       0.6       , 1.33333333, 5.        , 6.        , 0.25      ,
       0.28571429, 1.2       , 7.        , 0.125     , 0.42857143,
       1.4       , 0.375     , 2.33333333, 3.5       , 0.14285714,
       0.8       , 0.22222222, 1.75      , 0.3       , 1.25      ,
       0.11111111, 0.57142857, 0.83333333, 1.16666667, 0.85714286,
       0.71428571, 0.875     , 0.625     , 0.1       , 0.08333333,
       0.7       , 0.09090909, 0.27272727, 0.07142857, 0.15384615,
       0.23076923, 0.44444444, 0.07692308, 0.21428571, 0.13333333,
       0.36363636, 0.54545455, 0.55555556, 0.45454545, 0.1875    ,
       0.26666667, 0.0625    , 0.30769231, 0.06666667, 0.05263158,
       0.04761905, 0.18181818, 0.11764706, 0.05555556, 0.23529412,
       0.17647059, 0.21052632, 0.15789474, 0.05882353, 0.10526

In [10]:
import pandas as pd

# 1) 전체에 NaN이 하나라도 있는지 확인
has_nan = df.isna().values.any()
print("df에 NaN 존재 여부:", has_nan)

# 2) NaN이 포함된 행만 필터링
nan_rows = df[df.isna().any(axis=1)]

# 3) 결과 출력 (필요하면 .head()로 일부만)
print(f"NaN을 포함한 행 개수: {len(nan_rows)}")
display(nan_rows)
nan_columns = df.columns[df.isna().any()]
print("NaN을 포함한 컬럼들:", list(nan_columns))


df에 NaN 존재 여부: False
NaN을 포함한 행 개수: 0


,date,spot_id,vehicle_type,rev_per_vehicle_log1p,rev_per_vehicle,coupon_per_vehicle,avg_vehicle_age_inv,avg_standard_rate_inv,dow,week,...,land_price_log,region_total_rev,region_vehicle_count,region_rev_per_vehicle,region_code,doy_sin,doy_cos,month_sin,month_cos,region_vehicle_mean_rev


NaN을 포함한 컬럼들: []


In [11]:
df.columns

Index(['date', 'spot_id', 'vehicle_type', 'rev_per_vehicle_log1p',
       'rev_per_vehicle', 'coupon_per_vehicle', 'avg_vehicle_age_inv',
       'avg_standard_rate_inv', 'dow', 'week', 'month', 'quarter',
       'is_weekend', 'is_holiday', 'offday_run', 'day_to_offday', 'lag_1',
       'lag_7', 'lag_28', 'roll_mean_7', 'roll_std_7', 'inventory_est',
       'has_inventory', 'utilization_7', 'vehicle_count', 'coupon_count',
       'coupon_flag_prev1', 'coupon_count_lag7', 'T', 'RH', 'P', 'W', 'S',
       'weather_score', 'rain_mm_lag1', 'temp_lag1', 'pct_child', 'pct_teen',
       'pct_adult', 'pct_senior', 'land_price_log', 'region_total_rev',
       'region_vehicle_count', 'region_rev_per_vehicle', 'region_code',
       'doy_sin', 'doy_cos', 'month_sin', 'month_cos',
       'region_vehicle_mean_rev'],
      dtype='object')

In [8]:
hf_df

,unique_id,ds,y,region_code,spot_id,vehicle_type
0,1.0_Microcar,2023-01-06,25440.00,133,1.0,Microcar
20,2.0_Subcompact SUV,2023-01-06,5840.00,133,2.0,Subcompact SUV
38,7.0_Microcar,2023-01-06,18289.75,133,7.0,Microcar
57,9.0_Microcar,2023-01-06,41662.00,133,9.0,Microcar
59,9.0_Compact,2023-01-06,3280.00,133,9.0,Compact
...,...,...,...,...,...,...
25656770,4362.0_Midsize HEV,2024-12-31,0.00,143,4362.0,Midsize HEV
25656787,4390.0_Compact SUV,2024-12-31,0.00,119,4390.0,Compact SUV
25656804,4331.0_Compact,2024-12-31,28360.00,108,4331.0,Compact
25656816,4331.0_Electric,2024-12-31,76510.00,108,4331.0,Electric


In [26]:
from hierarchicalforecast.utils import aggregate

hierarchy_spec = [
    ['region_code'],                           # L2 ─ 가장 상위
    ['region_code', 'spot_id'],                # L1 ─ 지역별 스팟 합계
    ['region_code', 'spot_id', 'vehicle_type'] # L0 ─ 바텀(전체 컬럼)
]

# 1) df에 NaN unique_id가 생기지 않도록 vehicle_type·spot_id null 제거
hf_df = hf_df.dropna(subset=['spot_id','vehicle_type','region_code']) 

# 2) 그러고 나서 중복까지 제거
hf_df = hf_df.drop_duplicates(subset=['unique_id','ds'])   

# 3) aggregate 다시 실행
Y_hier_df, S, tags = aggregate(df=hf_df, spec=hierarchy_spec, id_col='_hier_id')


In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT
from neuralforecast.models import HINT 
from neuralforecast.losses.pytorch import GMM

# ① TFT 하이퍼파라미터
h  = 20   # prediction horizon (= max_prediction_length)
inp = 60  # encoder 길이 (= max_encoder_length)

quantiles = [0.025, 0.5, 0.975]                   # any set you need

tft = TFT(
    h=h, input_size=inp,
    hidden_size=64, dropout=0.1,
    learning_rate=1e-3, n_blocks=1,
    init_epochs=1, max_steps=30,
    random_state=42,
    loss=GMM(n_components=10, quantiles=quantiles)   # ★ key change
)

model = HINT(
    h=h, S=S, model=tft,
    reconciliation='MinTraceOLS'        # MinT-OLS
)

nf = NeuralForecast(models=[model], freq='D', accelerator='gpu')  # GPU 사용 명시 가능


Seed set to 1


InvalidIndexError: (slice(None, 1871, None), slice(None, None, None))

In [23]:
# 1) tags에서 bottom-level ID 리스트 가져오기
bottom_ids = tags[list(tags.keys())[-1]]      # e.g. tags['region_code/spot_id/vehicle_type']:contentReference[oaicite:0]{index=0}

# 2) df가 가진 real IDs
df_ids     = pd.Series(hf_df['unique_id'].unique())

# 3) 차집합 계산
extras = set(bottom_ids) - set(df_ids)       # Python set difference :contentReference[oaicite:1]{index=1}
print("Extra ID(s) in S but not in df:", extras)


Extra ID(s) in S but not in df: {'119/2134.0/Microcar', '119/793.0/Subcompact SUV', '119/3408.0/Midsize SUV', '108/510.0/Compact', '108/1934.0/Midsize HEV', '131/204.0/Microcar', '108/3355.0/Midsize HEV', '108/1883.0/Subcompact SUV HEV', '108/3504.0/Compact HEV', '119/2358.0/Subcompact', '108/2132.0/Subcompact SUV', '131/1094.0/Microcar', '119/3608.0/Midsize', '119/2089.0/Electric', '108/4210.0/Midsize SUV', '131/3960.0/Midsize HEV', '159/2098.0/Subcompact SUV', '119/3395.0/Midsize HEV', '131/210.0/Imported', '108/986.0/Microcar', '119/4298.0/Compact', '119/3289.0/Micro SUV', '119/2060.0/Passenger Van', '119/3317.0/Midsize', '108/4190.0/Compact', '133/897.0/Subcompact SUV', '108/4201.0/Upper Midsize', '119/3285.0/Electric', '108/3621.0/Midsize', '119/2306.0/Compact', '108/552.0/Compact', '143/2202.0/Midsize HEV', '119/2060.0/Compact HEV', '112/3580.0/Subcompact SUV HEV', '119/3861.0/Subcompact SUV HEV', '108/3686.0/Compact', '119/3908.0/Midsize HEV', '108/1452.0/Midsize SUV', '108/2377

In [ ]:
# 학습
nf.fit(hf_df)

# 예측 (h=20 스텝)
fcst = nf.predict()
fcst.head()


In [28]:
import logging

import pandas as pd
from utilsforecast.plotting import plot_series

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.utils import AirPassengersDF

In [29]:
logging.getLogger('pytorch_lightning').setLevel(logging.ERROR)

In [30]:
feat_path = Path("data/feature_store/daily_features_per.feather")
df = pd.read_feather(feat_path)

# ▶︎ 날짜·정렬은 이미 돼 있다고 가정
df["date"] = pd.to_datetime(df["date"])

In [33]:
Y_df=df.copy()

Y_train_df = Y_df[Y_df['date']<='2023-10-31'] # 132 train
Y_test_df = Y_df[Y_df['date']>'2023-10-31'] # 12 test

In [37]:
from neuralforecast.models import AutoTFT

ImportError: cannot import name 'AutoTFT' from 'neuralforecast.models' (/home/.venv/lib/python3.12/site-packages/neuralforecast/models/__init__.py)

In [ ]:
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoTFT(h=12, config=config, num_samples=1, cpus=1)
nf = NeuralForecast(models=models, freq='ME')
nf.fit(df=Y_train_df)
Y_hat_df = nf.predict()

Seed set to 1


MemoryError: Unable to allocate 18.3 TiB for an array with shape (2240310, 1120154) and data type float64